### In this project, we are working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `MessyMedicalData.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The main aim of this project is to correctly identify and then extract all of the different date variants encoded in this dataset. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

In order to do this project, assume the following:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.


### First the data is converted into a dataframe using pandas 

In [1]:
import pandas as pd

doc = []
with open('MessyMedicalData.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

### Since we are going to have a consistant data, we have used the following function to convert each month from word to digit

In [2]:
def month_converter(month):
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    return str(months.index(month) + 1)
    

#### In this part, we are going to extract date which are based on following patterns:

#### <center> 04/20/2009; 04/20/09; 4/20/09; 4/3/09 <center>

In [3]:
df1 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2,4})')
df1.drop(df1.index[73], axis=0,inplace=True)
df1.columns = ['month', 'day', 'year']
df1 = df1[['day', 'month', 'year']] 
print(df1.shape)
df1.head()

(125, 3)


,,day,month,year
,match,,,
0,0,25,03,93
1,0,18,6,85
2,0,8,7,71
3,0,27,9,75
4,0,6,2,96


#### In this part, we are going to extract date which are based on following patterns:

#### <center> Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009; <center>
    
#### <center> 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009; <center>

We have not used any '-' since we have founf that there is no date that has the following pattern: Mar-20-2009

In [4]:
df2 = df.str.extractall(r'(\d{1,2})[,]* ((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[.,]* (\d{2,4})')
df2.columns = ['day', 'month', 'year']
df2['month'] = pd.DataFrame(df2['month'].apply(lambda x: month_converter(x)))
print(df2.shape)
df2.head()

(69, 3)


,,day,month,year
,match,,,
125,0,24,1,2001
126,0,10,9,2004
127,0,26,5,1982
128,0,28,6,2002
129,0,06,5,1972


#### In this part, we are going to extract date which are based on following patterns:

#### <center> 2009 Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009 <center>

The main different of this part with the above one is using 1st, 2nd, 3th

In [5]:
df3 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[.]* (\d{1,2})[,thstnd]* (\d{2,4})')
df3.columns = ['month', 'day', 'year']
df3 = df3[['day', 'month', 'year']]
df3['month'] = pd.DataFrame(df3['month'].apply(lambda x: month_converter(x)))
print(df3.shape)
df3.head()

(34, 3)


,,day,month,year
,match,,,
194,0,11,4,1990
195,0,30,5,2001
196,0,18,2,1994
197,0,18,2,1981
198,0,11,10,2013


#### In this part, we are going to extract date which are based on following patterns:

#### <center> Feb 2009; Sep 2009; Oct 2010 <center>

In [6]:
df4 = df[228:343].str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[.,]* (\d{4})')
df4['day'] = 1 
df4.columns = ['month', 'year', 'day']
df4 = df4[['day', 'month', 'year']]
df4['month'] = pd.DataFrame(df4['month'].apply(lambda x: month_converter(x)))
print(df4.shape)
df4.head()

(115, 3)


,,day,month,year
,match,,,
228,0,1,9,1985
229,0,1,6,2011
230,0,1,5,1986
231,0,1,5,2016
232,0,1,7,1977


#### In this part, we are going to extract date which are based on following patterns:

#### <center> 6/2008; 12/2009 <center>

In [7]:
df5 = df[343:455].str.extractall(r'(\d{1,2})[/-](\d{4})')
df5['day'] = 1 
df5.columns = ['month', 'year', 'day']
df5 = df5[['day', 'month', 'year']]
print(df5.shape)
df5.head()

(112, 3)


,,day,month,year
,match,,,
343,0,1,6,1998
344,0,1,6,2005
345,0,1,10,1973
346,0,1,9,2005
347,0,1,03,1980


#### In this part, we are going to extract date which are based on following patterns:

#### <center> 2008; 2009 <center>

In [8]:
df6 = df[455:500].str.extractall(r'[^0-9](\d{4})[^0-9]')
df6 = pd.concat([df6, df.str.extractall(r'^(\d{4})[^0-9]')])
df6['day'] = 1
df6['month'] = 1
df6.columns = ['year', 'day', 'month']
df6 = df6[['day', 'month', 'year']]
print(df6.shape)
df6.head()

(45, 3)


,,day,month,year
,match,,,
455,0,1,1,1984
456,0,1,1,2000
457,0,1,1,2001
458,0,1,1,1982
459,0,1,1,1998


#### After extracting and normalising the data for different dates, here we concatnate them into a single dataframe

In [9]:
Final_df = pd.concat([df1, df2, df3, df4, df5, df6])

#### Since, some of the years are represented by 2-digit, we fixed them, by converting them to a 4-digit year

In [10]:
Final_df['year']=Final_df['year'].apply(lambda x: '19'+x if len(x)<=2 else x)

In [11]:
print(Final_df.shape)
Final_df.head()

(500, 3)


,,day,month,year
,match,,,
0,0,25,03,1993
1,0,18,6,1985
2,0,8,7,1971
3,0,27,9,1975
4,0,6,2,1996


### Now, to remove a level of index, the following code is used. Also the column types are converted to a int

In [12]:
Final_df.sort_index(axis = 0, inplace=True)

Final_df['year'] = Final_df['year'].astype(int)
Final_df['month'] = Final_df['month'].astype(int)
Final_df['day'] = Final_df['day'].astype(int)

In [13]:
Final_df = Final_df.reset_index()
#Final_df.drop(['index'], axis=1, inplace=True)

In [14]:
Final_df.columns

Index(['level_0', 'match', 'day', 'month', 'year'], dtype='object')

In [15]:
Final_df.drop(['level_0', 'match'], axis=1, inplace=True)

### <center> The final table is

In [16]:
print(Final_df.shape)
Final_df.head()

(500, 3)


,day,month,year
0,25,3,1993
1,18,6,1985
2,8,7,1971
3,27,9,1975
4,6,2,1996
